<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spatstat-playground" data-toc-modified-id="Spatstat-playground-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spatstat playground</a></span><ul class="toc-item"><li><span><a href="#Get-the-package" data-toc-modified-id="Get-the-package-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get the package</a></span></li><li><span><a href="#DPPs-with-stationnary-isotropic-kernels" data-toc-modified-id="DPPs-with-stationnary-isotropic-kernels-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>DPPs with stationnary isotropic kernels</a></span><ul class="toc-item"><li><span><a href="#Gaussian-kernel" data-toc-modified-id="Gaussian-kernel-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Gaussian kernel</a></span><ul class="toc-item"><li><span><a href="#Pair-correlation-function" data-toc-modified-id="Pair-correlation-function-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Pair correlation function</a></span></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Simulation</a></span></li><li><span><a href="#Estimation" data-toc-modified-id="Estimation-1.2.1.3"><span class="toc-item-num">1.2.1.3&nbsp;&nbsp;</span>Estimation</a></span></li><li><span><a href="#Kernel-parameter-estimation-for-a-sample" data-toc-modified-id="Kernel-parameter-estimation-for-a-sample-1.2.1.4"><span class="toc-item-num">1.2.1.4&nbsp;&nbsp;</span>Kernel parameter estimation for a sample</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Spatstat playground

**First session Friday 06/18/2021** https://github.com/For-a-few-DPPs-more/spatstat-interface

Let's take Determinantal Point Processes (DPPs) as a running example to play with the [`spatstat`](https://spatstat.org/) (Spatial Statistics) R package from Python!
More specifically, we'll perform both sampling and inference on DPPs using `spatstat`.

To do this, one can use the Python package [`rpy2`](https://rpy2.github.io/doc/v3.4.x/html/overview.html#installation) to ensure the interoperability with [`R`](https://www.r-project.org/).
In other words, `rpy2` allows us to call `R` from `Python`.

`spatstat` has been split into multiple subpackages and extensions, see [`spatstat` GitHub repo](https://github.com/spatstat/spatstat).
In this notebook will make use of
- [spatstat.core](https://www.rdocumentation.org/packages/spatstat.core/versions/2.2-0)
- [spatstat.geom](https://www.rdocumentation.org/packages/spatstat.core/versions/2.2-0)

## Get the package

See the [Installation](https://github.com/For-a-few-DPPs-more/spatstat-interface/blob/main/README.md#installation) section on [GitHub](https://github.com/For-a-few-DPPs-more/spatstat-interface/)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

import rpy2.robjects as robjects
# Activate automatic conversion of numpy floats and arrays to corresponding R objects
from rpy2.robjects import numpy2ri
numpy2ri.activate() #numpy2ri.deactivate()

In [ ]:
from spatstat_interface.utils import to_pandas_data_frame
from spatstat_interface.interface import SpatstatInterface

spatstat = SpatstatInterface(update=True)
spatstat.import_package("core", "geom", update=True)
spatstat.core.__version__ # 2.3-0

In [ ]:
spatstat.core.__version__

## DPPs with stationnary isotropic kernels

### Gaussian kernel

$K(x, y) = \rho \exp(−\|\frac{x - y}{\alpha}\|^2)$

$\rho_{\max} = \left(\sqrt{\pi} \alpha\right)^{-d}$

In [ ]:
rho = 100
alpha = 0.05
d = 2
rho_max = (np.sqrt(np.pi) * alpha)**(-d)
assert rho <= rho_max

# spatstat.core.dppGauss(lambda=, alpha=, d=)
# However lambda is a reserved Python keyword
# Let's circumvent the problem using a dictionnary
params = {"lambda": rho, "alpha": alpha, "d": d}
my_dpp = spatstat.core.dppGauss(**params)
my_dpp

#### Pair correlation function

##### Theoretical pcf

In [ ]:
pcf = spatstat.core.pcfmodel(my_dpp)

In [ ]:
numpy2ri.activate()  # to call pcf on a numpy array
fig, ax = plt.subplots()
r = np.linspace(0, 0.3, 1000)
ax.plot(r, pcf(r))
ax.set_xlim(0, 0.15)

#### Simulation

Define the window where the points will be sampled from

In [ ]:
# using spatstat.geom.boxx
numpy2ri.activate()
bound = np.array([0, 2])
window = spatstat.geom.boxx(bound, bound)

# using spatstat.geom.owin
bound = robjects.FloatVector([0, 2])
window = spatstat.geom.owin(xrange=bound, yrange=bound)

Generate the sample using `spatstat.core.simulate_dppm`

In [ ]:
rsample = spatstat.core.simulate_dppm(my_dpp, W=window)
rsample

Convert spatstat sample to a numpy.array (as if the sample was generated or imported using Python)

In [ ]:
sample = np.array([rsample.rx2("x"), rsample.rx2("y")])
sample.shape

Display the sample

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
r = np.linspace(0, 0.3, 1000)
ax.scatter(*sample) # equivalent to scatter(sample[0], sample[1])
ax.set_aspect('equal', 'box')

#### Estimation

From a sample generated by spatstat

In [ ]:
# If numpy2ri.activate() the output of spatstat.core.pcf is automatically converted to a numpy array 
# Otherwise it is an R DataFrame that we can convert to a pandas DataFrame
numpy2ri.deactivate()
pcf_r = spatstat.core.pcf_ppp(rsample)
pcf_df = to_pandas_data_frame(pcf_r)
pcf_df

In [ ]:
fig, ax = plt.subplots()
r = np.linspace(0, 0.3, 1000)
numpy2ri.activate()
ax.plot(r, pcf(r))
pcf_df.plot("r", "iso", ax=ax)
ax.set_xlim(0, 0.15)

From a sample stored in a numpy array

In [ ]:
numpy2ri.activate()
# Each row of sample is associated to a coordinate, i.e., samples are stored columnwise
X = spatstat.geom.ppp(*sample, window=window)
numpy2ri.deactivate()
pcf_r = spatstat.core.pcf_ppp(X)
pcf_df = to_pandas_data_frame(pcf_r)
pcf_df

In [ ]:
fig, ax = plt.subplots()
r = np.linspace(0, 0.3, 1000)
numpy2ri.activate()
ax.plot(r, pcf(r))
pcf_df.plot("r", "iso", ax=ax)
ax.set_xlim(0, 0.15)

#### Kernel parameter estimation for a sample

In [ ]:
def convert_to_dict(x):
    return dict(zip(x.names, x))

Recall the parameters of the original DPP

In [ ]:
params = convert_to_dict(my_dpp.rx2("fixedpar"))
params

Fit a DPP from a sample

In [ ]:
dpp_family = spatstat.core.dppGauss
formula = robjects.Formula("X ~ 1")
formula.environment["X"] = rsample

my_dpp_fitted = spatstat.core.dppm(formula=formula, family=dpp_family).rx2("fitted")
fitted_params = convert_to_dict(my_dpp_fitted.rx2("fixedpar"))
fitted_params

In [ ]:
pcf_fitted_dpp = spatstat.core.pcfmodel(my_dpp_fitted)

Let's have a look at the correlation function.

In [ ]:
fig, ax = plt.subplots()

r = np.linspace(0, 0.3, 1000)
numpy2ri.activate()
ax.plot(r, pcf(r), label="original dpp")
pcf_df.plot("r", "iso", ax=ax, label="empirical")
ax.plot(r, pcf_fitted_dpp(r), label="fitted dpp")

ax.set_xlim(0, 0.15)
plt.legend()